In [1]:
import datetime
import os

import numpy as np
import pandas as pd

In [2]:
where_to_FIND = r"D:\VINHTRUONG_PHD\INSAR\202__GPS_Data\2__GPS_Sinica_2023\GNSS資料"
where_to_SAVE = r"D:\VINHTRUONG_PHD\INSAR\202__GPS_Data\2__GPS_Sinica_2023\Converted_GNSS"

In [3]:
def datetime_converter(RAW_YEAR):
    """
    RAW_YEAR: Danh sách số ngày cần chuyển đổi
    type(RAW_YEAR) : list
    """

    converted_time = []
    from datetime import datetime

    from dateutil.relativedelta import relativedelta

    for raw_value in RAW_YEAR:
        # Số ngày trong một năm, số liệu từ Wikipedia
        # Days in a year (source: Wikipedia)
        MULTIPLIER = 365.242199

        # Năm :  Lấy phần nguyên của số liệu năm
        # Nếu là năm nhận thì sẽ có 366 ngày

        # Leap Year -> 366 days
        YEAR = int(raw_value)
        if YEAR % 4 == 0:
            MULTIPLIER = 366

        # Số ngày : Lấy phần lẻ nhân với 365 (hoặc 366)
        DAYS = (raw_value % 1) * MULTIPLIER
        DAYS = int(DAYS // 1)

        # START_TIME là ngày 1 tháng 1, sau đó cộng cho số ngày quy đổi ở trên để ra số ngày ghi nhận số liệu
        START_TIME = datetime(YEAR, 1, 1)
        END_TIME = START_TIME + relativedelta(days=+DAYS)

        DATETIME = END_TIME.strftime("%Y/%m/%d")
        converted_time.append(DATETIME)

    return converted_time

In [4]:
READFOLDER = where_to_FIND
WRITEFOLDER = where_to_SAVE

FILELIST = []
SAVELIST = []

extension = ".neu_rmoffEQ"
for FILENAME in os.listdir(READFOLDER):
    #     if FILENAME.endswith('.txt'):
    if FILENAME.endswith(extension):
        FILEPATH = os.path.join(READFOLDER, FILENAME).replace("\\", "/")
        FILESAVE = os.path.join(WRITEFOLDER, FILENAME).replace("\\", "/")
        FILELIST.append(FILEPATH)
        SAVELIST.append(FILESAVE)

In [5]:
for FILEPATH, SAVEPATH in zip(FILELIST, SAVELIST):
    try:
        DATA = pd.read_table(FILEPATH, header=None, sep="\s+")

        DATA.columns = ["Datetime", "dN", "dE", "dU", "sN", "sE", "sU"]

        RAW_YEAR = DATA["Datetime"]

        CONVERTED = datetime_converter(RAW_YEAR)

        dataframe = pd.DataFrame(data=None, index=None, dtype=None)
        dataframe["Original"] = DATA["Datetime"]
        dataframe["Converted"] = np.array(CONVERTED, dtype=str)
        dataframe["dN(m)"] = DATA["dN"]
        dataframe["dE(m)"] = DATA["dE"]
        dataframe["dU(m)"] = DATA["dU"]
        dataframe["sN(m)"] = DATA["sN"]
        dataframe["sE(m)"] = DATA["sE"]
        dataframe["sU(m)"] = DATA["sU"]

        #     SAVENAME = SAVEPATH.replace('.txt', '.csv')
        SAVENAME = SAVEPATH.replace(extension, ".csv")
        dataframe.to_csv(SAVENAME, header=True, index=False, index_label=None)
        # print('Finish {}'.format(FILENAME))
        # print('Have been written as {}'.format(SAVENAME))
        # print()
    except Exception as e:
        print(os.path.basename(FILEPATH))
        print(e)
        pass